In [ ]:
import cv2
import numpy as np
import csv

video_path = '/Users/zzze/Downloads/Q5_6644_argue_and_door_slam.mp4'
cap = cv2.VideoCapture(video_path)

def detect_shapes(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(blurred, 50, 150)
    contours, _ = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    shapes = []
    for contour in contours:
        if cv2.contourArea(contour) > 50:
            approx = cv2.approxPolyDP(contour, 0.04 * cv2.arcLength(contour, True), True)
            if len(approx) == 3:
                shape_type = "Triangle"
            elif len(approx) == 4:
                shape_type = "Quadrilateral"
            elif len(approx) > 4:
                shape_type = "Circle"
            else:
                shape_type = "Unknown"
            shapes.append((contour, shape_type))
    return shapes

class BoundaryTracker:
    def __init__(self):
        self.nextObjectID = 0
        self.objects = {}
        self.disappeared = {}
        self.prev_frame = None
        self.lk_params = dict(winSize=(15, 15), maxLevel=2,
                              criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

    def register(self, contour, shape_type):
        self.objects[self.nextObjectID] = (contour, shape_type)
        self.disappeared[self.nextObjectID] = 0
        self.nextObjectID += 1

    def deregister(self, objectID):
        del self.objects[objectID]
        del self.disappeared[objectID]

    def update(self, frame, inputContours):
        if self.prev_frame is None:
            self.prev_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            for contour, shape_type in inputContours:
                self.register(contour, shape_type)
            return self.objects

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        if len(inputContours) == 0:
            for objectID in list(self.disappeared.keys()):
                self.disappeared[objectID] += 1
                if self.disappeared[objectID] > 50:
                    self.deregister(objectID)
            self.prev_frame = gray_frame
            return self.objects

        if len(self.objects) == 0:
            for contour, shape_type in inputContours:
                self.register(contour, shape_type)
        else:
            objectIDs = list(self.objects.keys())
            objectContours = [obj[0] for obj in self.objects.values()]

            if len(objectContours) > 0:
                prev_points = np.array([np.mean(contour, axis=0)[0] for contour in objectContours], dtype=np.float32).reshape(-1, 1, 2)
                new_points, status, error = cv2.calcOpticalFlowPyrLK(self.prev_frame, gray_frame, prev_points, None, **self.lk_params)

                usedRows = set()
                usedCols = set()

                for i, (new, old) in enumerate(zip(new_points, prev_points)):
                    if status[i] == 1:
                        dx, dy = new[0] - old[0]
                        objectID = objectIDs[i]
                        contour = objectContours[i] + [dx, dy]
                        self.objects[objectID] = (contour, self.objects[objectID][1])
                        self.disappeared[objectID] = 0
                        usedRows.add(i)
                        usedCols.add(i)

                unusedCols = set(range(0, len(new_points))).difference(usedCols)

                for i in unusedCols:
                    contour, shape_type = inputContours[i]
                    self.register(contour, shape_type)

        self.prev_frame = gray_frame
        return self.objects

def points_inside_contour(contour):
    mask = np.zeros((frame.shape[0], frame.shape[1]), dtype=np.uint8)
    cv2.drawContours(mask, [contour], -1, 255, -1)
    points = np.column_stack(np.where(mask == 255))
    return points

tracker = BoundaryTracker()
frame_count = 0
position_data = []
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    shapes = detect_shapes(frame)
    inputContours = []
    for contour, shape_type in shapes:
        inputContours.append((contour, shape_type))
        cv2.putText(frame, shape_type, tuple(contour[0][0]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        cv2.drawContours(frame, [contour], -1, (0, 255, 0), 2)

    objects = tracker.update(frame, inputContours)
    for (objectID, (contour, shape_type)) in objects.items():
        if len(contour) > 0:
            cv2.drawContours(frame, [contour], -1, (0, 255, 0), 2)
            points = points_inside_contour(contour)
            for point in points:
                position_data.append([frame_count, objectID, point[1], point[0]])

    cv2.imshow("Frame", frame)
    frame_count += 1
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

with open('position_data.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Frame", "ObjectID", "X", "Y"])
    writer.writerows(position_data)

print("Position data saved to position_data.csv")


error: OpenCV(4.10.0) /private/var/folders/qj/srrkczq5047d9pqpz3ssmcsm0000gn/T/pip-install-k0849zfc/opencv-python_159c02c3a11c40479ded816fc3c17bfd/opencv/modules/imgproc/src/drawing.cpp:2504: error: (-215:Assertion failed) npoints > 0 in function 'drawContours'
